In [173]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import scipy

movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [174]:
movie_texts = movies['overview'].apply(str).to_list()
movie_titles = movies['title'].apply(str).to_list()
text_tokens = []
for i in range(len(movie_texts)):
    text = movie_titles[i] + " " + movie_texts[i]
    text_tokens.append(gensim.utils.simple_preprocess(text))

tagged_text = [TaggedDocument(t, [i]) for i, t in enumerate(text_tokens)]

In [175]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=20)

model.build_vocab(tagged_text)

In [160]:
model.train(tagged_text, total_examples=model.corpus_count, epochs=model.epochs)

In [161]:
model.corpus_total_words

2494618

In [162]:
vecs = model.dv

In [163]:
print(vecs[1])

[ 0.05046481 -0.09876183  0.03245487 -0.3777033  -0.1982141  -0.22739336
  0.32246408  0.04032854 -0.55991495 -0.16753763  0.13530967 -0.2196553
 -0.10244115  0.22467586  0.02019576  0.052859    0.14084603 -0.24335442
 -0.5390076   0.13230218  0.2973383   0.25094438  0.44818303  0.44696388
  0.40981796 -0.04009291 -0.7710485  -0.05028486 -0.5707527   0.82427263
  0.39869824  0.4687731   0.5390988  -0.41483587 -0.30768448  0.17356251
 -0.1493999  -0.10809475  0.25890625 -0.03915172  0.09533122  0.11768537
 -0.10309069 -0.08330884  0.00668132 -0.00777857 -0.1768896  -0.06922936
  0.01848018  0.3474796 ]


In [164]:
model.wv.most_similar('good')

[('bad', 0.6349313855171204),
 ('easy', 0.5979958176612854),
 ('trouble', 0.5788624286651611),
 ('much', 0.5687113404273987),
 ('right', 0.5685654878616333),
 ('nice', 0.5620800256729126),
 ('really', 0.5617133975028992),
 ('simple', 0.5563761591911316),
 ('bit', 0.5502724051475525),
 ('big', 0.5465793609619141)]

In [165]:
ranks = []
second_ranks = []
for t in range(len(tagged_text)):
    inferred_vector = model.infer_vector(tagged_text[t].words)
    sims = model.dv.most_similar([inferred_vector], topn=100)
    try:
        rank = [docid for docid, sim in sims].index(t)
        ranks.append(rank)
    except ValueError:
        ranks.append(-1)
    second_ranks.append(sims[1])

In [166]:
import collections

counter = collections.Counter(ranks)
print(counter)
print(counter[0]/len(tagged_text))

Counter({0: 44314, -1: 393, 1: 168, 2: 54, 3: 35, 4: 29, 5: 18, 6: 18, 7: 17, 12: 16, 19: 15, 11: 13, 8: 12, 15: 12, 20: 10, 25: 10, 9: 9, 16: 9, 18: 9, 39: 9, 37: 8, 14: 8, 33: 8, 32: 8, 42: 7, 10: 7, 28: 7, 40: 7, 36: 7, 22: 7, 30: 7, 26: 6, 13: 6, 29: 6, 84: 6, 61: 6, 35: 6, 23: 6, 21: 6, 64: 5, 17: 5, 57: 5, 41: 5, 50: 5, 47: 5, 27: 5, 49: 5, 46: 5, 78: 5, 53: 4, 94: 4, 44: 4, 24: 4, 87: 4, 43: 4, 34: 4, 38: 4, 83: 3, 96: 3, 55: 3, 76: 3, 58: 3, 45: 3, 65: 3, 81: 3, 93: 3, 54: 3, 62: 3, 59: 3, 68: 3, 91: 3, 71: 2, 85: 2, 99: 2, 67: 2, 77: 2, 72: 2, 31: 2, 73: 2, 92: 2, 79: 2, 86: 2, 70: 2, 88: 2, 60: 2, 75: 2, 51: 1, 98: 1, 56: 1, 48: 1, 63: 1, 52: 1, 90: 1, 69: 1, 95: 1, 89: 1, 74: 1, 82: 1, 66: 1})
0.9746623850789601


In [167]:
ratings_train, ratings_test = train_test_split(ratings_small, test_size=0.3)
ratings_train_merge = ratings_train.merge(links_small, on='movieId')
ratings_train_merge.head()
ratings_test_merge = ratings_test.merge(links_small, on='movieId')
ratings_test_merge.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,196,2762,5.0,959223610,167404,745.0
1,231,2762,5.0,977005495,167404,745.0
2,115,2762,4.0,1107034350,167404,745.0
3,627,2762,4.0,1201381076,167404,745.0
4,644,2762,3.0,944933544,167404,745.0


In [168]:
# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
ram = 12    # in GB
count_to_big = 0
count_false_int = 0

# get all movie vector rated from user
def get_user_rated_movies(ids):
    
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                movie_vector = vecs[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                print("ValueError")


        #movie_vectors = scipy.sparse.vstack(movie_vector_list)
        movie_vectors = movie_vector_list
        return movie_vectors


In [169]:
userid_list = ratings_train['userId'].unique()
all_user_profiles = {}
for uid in userid_list:
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    user_rated_movies_vector_array = np.array(user_rated_movies_vector_list)
    user_vector = user_rated_movies_vector_array.sum(axis=0)
    #user_vector = np.mean(user_rated_movies_vector_array, axis=0)
    all_user_profiles[uid] = user_vector

ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError
ValueError

In [170]:
recommendation_users = {}
for uid in userid_list:
    sims = model.docvecs.most_similar([all_user_profiles[uid]], topn=10000)
    recommendation_users[uid] = sims
len(recommendation_users)

671

In [171]:
%%time
# NOT TRAIN TEST 
percentage_already_seen = []
for uid in userid_list:
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_tmdb_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            user_tmdb_list.append(int(tmdbId))
        except ValueError:
            print("ValueError: " + str(tmdbId))
    
    m_count = 0
    for mid, cos in user_recommendation[:len(user_profile)]:
        try:
            if int(mid) in user_tmdb_list:
                m_count += 1
        except ValueError:
            print("ValueError: " + str(mid) + " not in tmdbid list")    

    if m_count == len(user_profile):
        print(str(uid) + " : the first movies are already viewed")
    else:
        print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

    p = m_count/len(user_profile)
    percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))

0 of 22 in first recommended already seen
3 of 341 in first recommended already seen
ValueError: 607   NaN
Name: tmdbId, dtype: float64
ValueError: 5826   NaN
Name: tmdbId, dtype: float64
36 of 1700 in first recommended already seen
0 of 222 in first recommended already seen
3 of 645 in first recommended already seen
0 of 192 in first recommended already seen
0 of 51 in first recommended already seen
0 of 25 in first recommended already seen
ValueError: 8045   NaN
Name: tmdbId, dtype: float64
ValueError: 8505   NaN
Name: tmdbId, dtype: float64
2 of 229 in first recommended already seen
5 of 547 in first recommended already seen
3 of 513 in first recommended already seen
2 of 323 in first recommended already seen
31 of 1291 in first recommended already seen
0 of 77 in first recommended already seen
0 of 23 in first recommended already seen
ValueError: 607   NaN
Name: tmdbId, dtype: float64
ValueError: 2282   NaN
Name: tmdbId, dtype: float64
67 of 1868 in first recommended already seen
2

In [172]:
# vector size = 50
Average Percentage of already seen movies: 0.0024270959347025794
# vector size = 100, epochs = 10
Average Percentage of already seen movies: 0.0025281050916012454
# vector size = 100, epochs = 20
Average Percentage of already seen movies: 0.0016255834501928675
# vector size = 100, epochs = 20, count_min = 1
Average Percentage of already seen movies: 0.0014803785890319565
# vector_size=50, min_count=5, epochs=20
Average Percentage of already seen movies: 0.0015261212643458547
# vector_size=50, min_count=5, epochs=20, sum()

SyntaxError: invalid syntax (<ipython-input-172-61ab19dec88f>, line 2)